notebook for eda2: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/eda2_single_user.ipynb

In [4]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus

import itertools
from library.sb_utils import save_file

In [ ]:
df = pd.read_csv('../data/processed/full_data_cleaned.csv')
df.info()

In [3]:
# Overall goal: look for key products or clusters of products
# First, import all the dictionaries holding connections
# Among product, aisle, and department names and ids

